# 当代人工智能实验一：文本分类
## ——TF-IDF & TextCNN

### 一. 引入必要模块
numpy将用于数据的处理。
time用于记录代码运行时间。
train_test_split用于进行训练集与验证集的划分。
tensorflow.keras用于进行Tokenizer分词与TextCNN的训练。

In [1]:
import numpy as np
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

### 二. 载入数据

In [43]:
y = np.load('labelList.npy') # 标签
X = np.load('textVectorList.npy') # 向量化的文字

### 三. 利用训练集数据训练TF-IDF下的TextCNN模型
使用tensorflow.keras中的Sequential模块。
在这里，我们通过交叉验证来验证模型的正确性。

In [44]:
start_time = time.time()
model = Sequential()
model.add(Conv1D(128, 5, activation='relu', input_shape=(X.shape[1], 1)))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='softmax'))
# 编译模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 训练模型
model.fit(X.reshape(X.shape[0], X.shape[1], 1), y, epochs=5, validation_split=0.125)
# 评估模型
loss, accuracy = model.evaluate(X.reshape(X.shape[0], X.shape[1], 1), y)
end_time = time.time()
print("精确度为:", accuracy)
print("运行时间为:", end_time - start_time)

Epoch 1/5
219/219 [==============================] - 233s 1s/step - loss: 2.2308 - accuracy: 0.1193 - val_loss: 4.2754 - val_accuracy: 0.0000e+00
Epoch 2/5
219/219 [==============================] - 223s 1s/step - loss: 2.1904 - accuracy: 0.1381 - val_loss: 5.2808 - val_accuracy: 0.0000e+00
Epoch 3/5
219/219 [==============================] - 222s 1s/step - loss: 2.1770 - accuracy: 0.1487 - val_loss: 5.8710 - val_accuracy: 0.0000e+00
Epoch 4/5
219/219 [==============================] - 226s 1s/step - loss: 2.1656 - accuracy: 0.1533 - val_loss: 6.3031 - val_accuracy: 0.0000e+00
Epoch 5/5
250/250 [==============================] - 81s 323ms/step - loss: 2.7198 - accuracy: 0.1406
精确度为: 0.140625
运行时间为: 1215.3077547550201


我们发现，运行速度极其缓慢，且精确度也极低。这可能是因为TF-IDF的向量维度过高，且TF-IDF矩阵过于稀疏导致的。
在这里，我们尝试使用keras模块中的Tokenizer分词器进行分词与向量映射。

In [16]:
X = np.load("rawList.npy") # 文字
Y = np.load("labelList.npy") # 标签

In [17]:
start_time = time.time()
# 将文本转换为序列
# 设置词汇表大小
max_words = 50000
# 设置向量最大长度
maxlen = 200
# 使用Tokenizer进行文本转换为向量的操作
# 可以过滤大量标点符号
tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n')
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=maxlen)
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.125, random_state=42)
# 构建模型
model = Sequential()
model.add(Embedding(max_words, 128, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='softmax'))
# 编译模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 训练模型
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
end_time = time.time()
print("精确度为:", accuracy)
print("运行时间为:", end_time - start_time)

Epoch 1/5
219/219 [==============================] - 24s 107ms/step - loss: 1.5172 - accuracy: 0.5624 - val_loss: 0.5690 - val_accuracy: 0.8300
Epoch 2/5
219/219 [==============================] - 21s 97ms/step - loss: 0.3892 - accuracy: 0.8781 - val_loss: 0.3243 - val_accuracy: 0.8900
Epoch 3/5
219/219 [==============================] - 21s 96ms/step - loss: 0.1513 - accuracy: 0.9594 - val_loss: 0.2465 - val_accuracy: 0.9080
Epoch 4/5
219/219 [==============================] - 21s 95ms/step - loss: 0.0482 - accuracy: 0.9933 - val_loss: 0.2257 - val_accuracy: 0.9210
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2176 - accuracy: 0.9280
精确度为: 0.9279999732971191
运行时间为: 110.25250554084778


观察到，运行时间大幅度减少，且精确度较高，可以到约93%的水平。

### 四. 预测测试集结果
预测并观察最终的结果。

In [18]:
# 载入测试集
new_data = np.load("result_text.npy")
new_sequences = tokenizer.texts_to_sequences(new_data)
new_X = pad_sequences(new_sequences, maxlen=maxlen)

# 使用训练好的模型进行预测
predictions = model.predict(new_X)

# 如果模型是多类别分类的，你可能需要将预测的结果转换成类别标签
predicted_labels = np.argmax(predictions, axis=1)

63/63 [==============================] - 0s 6ms/step


In [19]:
predicted_labels

array([6, 4, 4, ..., 9, 6, 9], dtype=int64)